In [32]:
from gensim.models import KeyedVectors
model = 'full_model_OTAR_200d_10_iter.bin'
w2v_path = '/mnt/droplet/nfs/gns/literature/Santosh_Tirunagari/Models/'+model

w2v = KeyedVectors.load_word2vec_format(w2v_path, binary=True)


In [33]:
w1 = "cancer"

print("Most similar to {0}".format(w1), w2v.most_similar(positive=w1))

Most similar to cancer [('cancers', 0.7871127724647522), ('carcinoma', 0.7415306568145752), ('adenocarcinoma', 0.7244918346405029), ('carcinomas', 0.6808085441589355), ('tumors', 0.6718302965164185), ('tumor', 0.6668273210525513), ('melanoma', 0.601204514503479), ('adenocarcinomas', 0.5992950797080994), ('Cancer', 0.5688608884811401), ('cancer-derived', 0.5614933371543884)]


In [35]:
w1 = "LOXL1"
w2 = 'carcinoma'
w2v.similarity(w1, w2)

0.03829912902773729

In [36]:
w1 = "k-ras"

print("Most similar to {0}".format(w1), w2v.wv.most_similar(positive=w1))

Most similar to k-ras [('K-ras', 0.7236489653587341), ('c-K-ras', 0.6938831806182861), ('Ki-ras', 0.659921407699585), ('KRAS2', 0.6511437892913818), ('c-Ki-ras', 0.6445430517196655), ('K-ras-2', 0.6400061249732971), ('ki-ras', 0.6103998422622681), ('H-ras', 0.6081706285476685), ('braf', 0.5968808531761169), ('K-RAS', 0.5725597739219666)]


/home/stirunag/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [38]:
w1 = "Gene"

print("Most similar to {0}".format(w1), w2v.wv.most_similar(positive=w1))

Most similar to Gene [('Genes', 0.6356513500213623), ('gene', 0.6103031039237976), ('genes', 0.5458037853240967), ('Name', 0.5375816822052002), ('Microarray', 0.5367627739906311), ('3Gene', 0.5335216522216797), ('Transcriptional', 0.5265238881111145), ('Transcriptome', 0.49877551198005676), ('Fold', 0.4966423213481903), ('HEYL', 0.49448472261428833)]


/home/stirunag/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [39]:
result = w2v.most_similar(positive=['father', 'girl'], negative=['boy'])
print("{}: {:.4f}".format(*result[0]))

cousin: 0.7532


In [40]:
result = w2v.most_similar(positive=['IL6', 'Diabetes'], negative=['Cancer'])
print("{}: {:.4f}".format(*result[0]))

IL1: 0.5179


In [43]:
result = w2v.most_similar(positive=['cancer', 'TNF'], negative=['LOXL1'])
print("{}: {:.4f}".format(*result[0]))

TNFÎ±: 0.4472


In [45]:
import nltk
from nltk.corpus import stopwords

def clean_my_text(full_text):
    # stopset = set(stopwords.words('english'))  # | set(string.punctuation)
    tokens = nltk.word_tokenize(full_text)
    cleanup = [token for token in tokens if len(token) > 2]
    return cleanup

In [44]:
sentence = 'To evaluate serum glutathione S-transferase B1 (GST B1), a highly sensitive test of hepatocellular function, as a means of identifying liver disease in patients with cystic fibrosis (CF).'
w2v.doesnt_match(clean_my_text(sentence))

'hepatocellular'

In [46]:
keys = ['LOXL1', 'cancer', 'cystic']

embedding_clusters = []
word_clusters = []
for word in keys:
    embeddings = []
    words = []
    for similar_word, _ in w2v.most_similar(word, topn=30):
        words.append(similar_word)
        embeddings.append(w2v[similar_word])
    embedding_clusters.append(embeddings)
    word_clusters.append(words)
    
    

In [47]:
from sklearn.manifold import TSNE
import numpy as np

embedding_clusters = np.array(embedding_clusters)
n, m, k = embedding_clusters.shape
tsne_model_en_2d = TSNE(perplexity=15, n_components=2, init='pca', n_iter=3500, random_state=32)
embeddings_en_2d = np.array(tsne_model_en_2d.fit_transform(embedding_clusters.reshape(n * m, k))).reshape(n, m, 2)


In [48]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
% matplotlib inline


def tsne_plot_similar_words(title, labels, embedding_clusters, word_clusters, a, filename=None):
    plt.figure(figsize=(16, 9))
    colors = cm.rainbow(np.linspace(0, 1, len(labels)))
    for label, embeddings, words, color in zip(labels, embedding_clusters, word_clusters, colors):
        x = embeddings[:, 0]
        y = embeddings[:, 1]
        plt.scatter(x, y, c=color, alpha=a, label=label)
        for i, word in enumerate(words):
            plt.annotate(word, alpha=0.5, xy=(x[i], y[i]), xytext=(5, 2),
                         textcoords='offset points', ha='right', va='bottom', size=8)
    plt.legend(loc=4)
    plt.title(title)
    plt.grid(True)
    if filename:
        plt.savefig(filename, format='png', dpi=150, bbox_inches='tight')
    plt.show()


tsne_plot_similar_words('Similar words from OTAR', keys, embeddings_en_2d, word_clusters, 0.7,
                        'similar_words_OTAR.png')

UsageError: Line magic function `%` not found.


In [51]:
from collections import defaultdict
import os

wordcount = defaultdict(int)

# get the current working directory and file
data_dir_path = '/mnt/droplet/nfs/gns/literature/Santosh_Tirunagari/Dataset/'

for fname in os.listdir(data_dir_path):
    print(fname)
    for line in open(os.path.join(data_dir_path, fname), 'r', encoding='ISO-8859-1'):
        for word in clean_my_text(line):
            wordcount[word] += 1

In [52]:
import itertools
import collections

w2cSorted=dict(sorted(wordcount.items(), key=lambda x: x[1],reverse=True))

In [53]:
x = itertools.islice(w2cSorted.items(), 0, 10)

for key, value in x:
    print(key, value)

In [54]:
import csv
newFile = '/home/stirunag/TF.txt'

def makeFile(textSorted, newFile) :
    from collections import Counter
    with open(newFile, "w") as f:
        wr = csv.writer(f,delimiter=" ")
        wr.writerows(Counter(textSorted).items())
        
        
makeFile(w2cSorted, newFile)


In [7]:
from gensim.models import KeyedVectors
model = 'ncbi_pmc2ena4ms_word2vec.model.wv.vectors.npy'
w2v_path = '/nfs/gns/literature/maaly/metagenome/python/scripts/ML/word2vec/'+model

w2v = KeyedVectors.load_word2vec_format(w2v_path, binary=True)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x93 in position 0: invalid start byte

In [1]:
from gensim.models import Word2Vec
import numpy
from gensim.models import FastText

In [2]:
model = 'ncbi_pmc2ena4ms_fasttext.model'
w2v_path = '/nfs/gns/literature/maaly/metagenome/python/scripts/ML/word2vec/'+model

model = FastText.load(w2v_path)

ModuleNotFoundError: No module named 'numpy.random._pickle'